In [0]:
pip install librosa

Python interpreter will be restarted.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Not uninstalling numpy at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2ac62241-9a95-414a-b9f4-8c06488685e0
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Not uninstalling threadpoolctl at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2ac62241-9a95-414a-b9f4-8c06488685e0
    Can't uninstall 'threadpoolctl'. No files were found to uninstall.
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Not uninstalling joblib at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2ac62241-9a95-414a-b9f4-8c06488685e0
    Can't uninstall 'joblib'. No files

In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import StructType, StructField, ArrayType, DoubleType, FloatType, StringType

import os
import librosa
import numpy as np
import pandas as pd

spark = SparkSession.builder.appName("FingerprintExtraction").getOrCreate()

In [0]:
from pyspark.sql.functions import input_file_name
import os

# Read list of MP3 files in Bronze layer
mp3_path = "dbfs:/FileStore/bronze/mp3/"

# Create DataFrame from file paths
df_mp3_files = spark.read.format("binaryFile").load(mp3_path)
df_mp3_files = df_mp3_files.withColumn("file_name", input_file_name())
df_mp3_files.select("file_name").show(truncate=False)

+----------------------------------------------------------------------------------------------+
|file_name                                                                                     |
+----------------------------------------------------------------------------------------------+
|dbfs:/FileStore/bronze/mp3/Kendrick_Lamar___Not_Like_Us.mp3                                   |
|dbfs:/FileStore/bronze/mp3/Chappell_Roan___Pink_Pony_Club__Official_Music_Video_.mp3          |
|dbfs:/FileStore/bronze/mp3/Lady_Gaga__Bruno_Mars___Die_With_A_Smile__Official_Music_Video_.mp3|
|dbfs:/FileStore/bronze/mp3/PARTYNEXTDOOR___DRAKE___NOKIA__Official_Lyric_Video_.mp3           |
|dbfs:/FileStore/bronze/mp3/Billie_Eilish___BIRDS_OF_A_FEATHER__Official_Music_Video_.mp3      |
|dbfs:/FileStore/bronze/mp3/Kendrick_Lamar___tv_off__Official_Audio_.mp3                       |
|dbfs:/FileStore/bronze/mp3/Teddy_Swims___Lose_Control__The_Village_Sessions_.mp3              |
|dbfs:/FileStore/bronze/mp3/Ke

Define Fingerprint Extraction Function

In [0]:
import librosa
import numpy as np

def extract_fingerprint_from_path(local_path, top_percentile=95):
    try:
        y, sr = librosa.load(local_path, sr=44100)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        threshold = np.percentile(D, top_percentile)
        peaks = np.argwhere(D > threshold)  # (freq_bin, time_bin)

        # Convert to flattened string format
        flattened = [f"{int(f)},{int(t)}" for f, t in peaks]
        return flattened
    except Exception as e:
        return [f"error: {str(e)}"]

Register Spark UDF

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

extract_fingerprint_udf = udf(extract_fingerprint_from_path, ArrayType(StringType()))

Apply Fingerprint Extraction

In [0]:
from pyspark.sql.functions import expr

# Convert Spark DBFS path to real file path for librosa
df_with_local_path = df_mp3_files.withColumn("local_path", expr("translate(file_name, 'dbfs:/', '/dbfs/')"))

# Apply fingerprint UDF
df_fingerprinted = df_with_local_path.withColumn("fingerprints", extract_fingerprint_udf("local_path"))

df_fingerprinted.select("file_name", "fingerprints").show(truncate=False)

+----------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|file_name                                                                                     |fingerprints                                                                                                                                  |
+----------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|dbfs:/FileStore/bronze/mp3/Kendrick_Lamar___Not_Like_Us.mp3                                   |[error: [Errno 2] No such file or directory: '/dbfs/FileStore/dronze/mp3/Ken/rick_Lamar___Not_Like_Uf.mp3']                                   |
|dbfs:/FileStore/bronze/mp3/Chappell_Roa

Explode Fingerprints & Column Cleaning

In [0]:
from pyspark.sql.functions import explode, split

df_flat = df_fingerprinted.withColumn("fingerprint", explode("fingerprints"))
df_flat = df_flat.withColumn("freq_bin", split("fingerprint", ",")[0].cast("int"))
df_flat = df_flat.withColumn("time_bin", split("fingerprint", ",")[1].cast("int"))

df_flat = df_flat.select("file_name", "freq_bin", "time_bin")
df_flat.show(truncate=False)

+----------------------------------------------------------------------------------------------+--------+--------+
|file_name                                                                                     |freq_bin|time_bin|
+----------------------------------------------------------------------------------------------+--------+--------+
|dbfs:/FileStore/bronze/mp3/Kendrick_Lamar___Not_Like_Us.mp3                                   |null    |null    |
|dbfs:/FileStore/bronze/mp3/Chappell_Roan___Pink_Pony_Club__Official_Music_Video_.mp3          |null    |null    |
|dbfs:/FileStore/bronze/mp3/Lady_Gaga__Bruno_Mars___Die_With_A_Smile__Official_Music_Video_.mp3|null    |null    |
|dbfs:/FileStore/bronze/mp3/PARTYNEXTDOOR___DRAKE___NOKIA__Official_Lyric_Video_.mp3           |null    |null    |
|dbfs:/FileStore/bronze/mp3/Billie_Eilish___BIRDS_OF_A_FEATHER__Official_Music_Video_.mp3      |null    |null    |
|dbfs:/FileStore/bronze/mp3/Kendrick_Lamar___tv_off__Official_Audio_.mp3        

Write Flattened Fingerprints

In [0]:
df_flat.write.mode("overwrite").format("delta").save("dbfs:/FileStore/silver/flattened_fingerprints/")

In [0]:
df_flat.write.mode("overwrite").format("delta").save("dbfs:/FileStore/silver/flattened_fingerprints/")
df_flat.write.mode("overwrite").option("header", True).csv("dbfs:/FileStore/silver/flattened_fingerprints_csv/")

In [0]:
spark.stop()